In [ ]:
import pandas as pd
import numpy as np
import os
import json
import sys


original_cwd = os.getcwd()

backend_path = os.path.abspath(os.path.join(original_cwd, "../.."))
print(backend_path)
print(sys.path)

if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

from shared.snowflake.client import SnowflakeClient
from experiments.utils.llm import build_prompt, get_llm_response, compare_ground_truth_vs_llm, count_number_tokens, split_docs_recursively

print(f"Returned to original working directory: {os.getcwd()}")

**laod the config file**

In [ ]:
if not os.path.exists("config"):
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

CONFIG_FILE_PATH = "config/config.json"

with open(CONFIG_FILE_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

**set the experience id** 

In [ ]:
EXPERIENCE_ID = config["experiments_specifique_params"]["experiment_id"]

print(EXPERIENCE_ID)

**load the embedding file**

In [ ]:
INPUT_EMBEDDINGS_FILE = config["output_recipies_embedding_file"].format(
    experiment_id=EXPERIENCE_ID
)
df_recipes_cleaned = pd.read_csv(INPUT_EMBEDDINGS_FILE)


emb_columns = [col for col in df_recipes_cleaned.columns if col.endswith('_EMB')]
print(f"Found embedding columns: {emb_columns}")

for col in emb_columns:
    df_recipes_cleaned[col] = df_recipes_cleaned[col].apply(
        lambda x: np.fromstring(x.strip('[]'), sep=' ', dtype=np.float32)
    )

for col in emb_columns:
    print(f"{col} -> first embedding shape: {df_recipes_cleaned[col][0].shape}")

**load the models**

In [ ]:
#load models

import torch
from sentence_transformers import SentenceTransformer
from torch.nn.functional import normalize

MODELS_CONFIG = config["models"]
COLUMNS_TO_EMBEDDE = config["columns_embedding"]

#create a dict {name model : model} 
MODELS_LIST = [SentenceTransformer(model_id) for model_id in MODELS_CONFIG]
MODEL_DICT = dict(zip(MODELS_CONFIG, MODELS_LIST))

**define function to retrive documents**

In [ ]:
def retrieve_documents(
    query: str,
    model: SentenceTransformer,
    documents: list,
    df: pd.DataFrame,
    columns_to_select: list,
    top_k: int
) -> dict:
    """
    Retrieve top-k documents relevant to the query using the specified embedding model.

    Returns:
        list: [{col1: ..., col2: ..., similarity_score: ...}, ...]
    """

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    # Generate query embedding
    query_embedding = model.encode([query], convert_to_tensor=True, device=device)
    query_embedding = normalize(query_embedding, p=2, dim=1)

    # Build document embedding tensor
    document_embeddings = torch.stack(
        [torch.tensor(doc, dtype=query_embedding.dtype, device=device) for doc in documents]
    )
    document_embeddings = normalize(document_embeddings, p=2, dim=1)

    # Compute cosine similarity
    cosine_scores = torch.matmul(query_embedding, document_embeddings.T).squeeze(0)

    # Top-k results
    top_k_results = torch.topk(cosine_scores, k=top_k)
    top_k_indices = top_k_results.indices.tolist()
    top_k_scores = top_k_results.values.tolist()

     # Select rows & columns
    retrieved_df = df.iloc[top_k_indices][columns_to_select].copy()

    # Add similarity scores
    retrieved_df["similarity_score"] = top_k_scores

    # Return list of documents
    return retrieved_df.to_dict(orient="records")


**get the top_k**

In [ ]:
TOP_K_LIST = config["experiments_specifique_params"]["top_k"]

TOP_K_LIST

**Get test queries**

In [ ]:
queries_test_file_path = config["query_test_file_path"]

with open(queries_test_file_path, 'r') as file:
    QUERIES_TEST_LIST = json.load(file)
    
QUERIES_TEST_LIST

**Get list of columns to select from the data**

In [ ]:
DATA_COLUMNS = config['data_columns']
DATA_COLUMNS

**For each top_k, model, embedding config, queries assign the doc retrived**

In [ ]:
# OUTPUT_DATA_DIR = config["output_retrieved_documents_file"].format(experiment_id=EXPERIENCE_ID)
# os.makedirs(OUTPUT_DATA_DIR, exist_ok=True)

OUTPUT_RETRIEVED_DOCS_FILE = config["output_retrieved_documents_file"].format(
    experiment_id=EXPERIENCE_ID    
)

# Check if the file exists
if os.path.exists(OUTPUT_RETRIEVED_DOCS_FILE):
    print(f"Loading cached results from {OUTPUT_RETRIEVED_DOCS_FILE} ...")
    with open(OUTPUT_RETRIEVED_DOCS_FILE, "r", encoding="utf-8") as f:
        retrived_model_query_docs_dict = json.load(f)
else:
    print("File not found. Running retrieval calculations ...")

    EMB_COLS = [col for col in df_recipes_cleaned.columns if col.endswith("_EMB")]
    retrived_model_query_docs_dict = {}

    for k in TOP_K_LIST:
        for emb_col in EMB_COLS:
            for query in QUERIES_TEST_LIST:

                # Convert embedding column to a list of numpy arrays
                documents = df_recipes_cleaned[emb_col].apply(np.array).to_list()

                # Build model name + config name from column name
                model_name = "/".join(emb_col.split("/")[:-1])
                config_name = emb_col.split("/")[-1].replace("_EMB", "")

                model = MODEL_DICT[model_name]

                retrieved_documents_list = retrieve_documents(
                    query=query,
                    model=model,
                    documents=documents,
                    df=df_recipes_cleaned,
                    columns_to_select=DATA_COLUMNS,
                    top_k=k
                )

                retrived_model_query_docs_dict \
                    .setdefault(k, {}) \
                    .setdefault(model_name, {}) \
                    .setdefault(config_name, {})[query] = retrieved_documents_list

    # Save to file for future runs
    with open(OUTPUT_RETRIEVED_DOCS_FILE, "w", encoding="utf-8") as f:
        json.dump(retrived_model_query_docs_dict, f, indent=4)

**For each query get the unique doc retrived**

In [ ]:
query_retrived_docs_dict = {}
seen = {}

for k_dict in retrived_model_query_docs_dict.values():
    for model_dict in k_dict.values():
        for config_dict in model_dict.values():
            for query, docs in config_dict.items():
                for doc in docs:
                    clean_doc = {col: doc[col] for col in DATA_COLUMNS}
                    doc_id = clean_doc['ID']
                    
                    if doc_id in seen.get(query, set()):
                        continue

                    seen.setdefault(query, set()).add(doc_id)
                    
                    query_retrived_docs_dict.setdefault(query, []).append(clean_doc)


In [ ]:
len(query_retrived_docs_dict['15-minute high-protein vegan lunch for office meal prep no microwave'])

In [ ]:
query_retrived_docs_dict.keys()

**LLM as judge to evaluate those retrived doc for each query**

**load the prompt**

In [ ]:
PROMPT_EVAL_PATH = config["eval_prompt_without_justification_file"]

with open(PROMPT_EVAL_PATH, "r") as f:
    prompt_template = f.read()

**load the LLM and its config**

In [ ]:
LLM_MODEL = config["llm_model"]
LLM_TEMPERATURE = config["temperature"]
LLM_MAX_TOKENS = config['max_tokens']
LLM_MODEL_CONTEXT_WINDOWS = config["context_window"] - LLM_MAX_TOKENS

**calculate the number of tokens in the prompt template**

In [ ]:
num_token_prompt_template = len(str(prompt_template)) // 3
num_token_prompt_template

**define the schema of the response**

In [ ]:
# JSON schema for recipe relevance ratings
json_schema = config['llm_json_format_without_justification']
json_schema

In [ ]:
MAX_RETRIES_PER_BATCH = 3

llm_results = []

# OUTPUT_DATA_DIR = config["output_data_dir"].format(experiment_id=EXPERIENCE_ID)
# os.makedirs(OUTPUT_DATA_DIR, exist_ok=True)

# OUTPUT_RETRIEVED_DOCS_RELEVENCE_FILE = config["output_retrived_documents_relevence_file"].format(
#     experiment_id=EXPERIENCE_ID    
# )

# # Check if the file exists
# if os.path.exists(OUTPUT_RETRIEVED_DOCS_RELEVENCE_FILE):
#     print(f"Loading cached results from {OUTPUT_RETRIEVED_DOCS_RELEVENCE_FILE} ...")
#     with open(OUTPUT_RETRIEVED_DOCS_RELEVENCE_FILE, "r", encoding="utf-8") as f:
#         llm_results = json.load(f)
# else:
# print("File not found. Running llm as judge calculations ...")

for query in query_retrived_docs_dict:
    docs = query_retrived_docs_dict[query]
    
    query_doc_relevance_dict = {
        "query_text": query,
        "relevance_judgments": []
    }
    
    
    num_token_query = len(str(query)) // 3
    num_token_docs = len(str(docs)) // 3
    
    doc_entries_list = split_docs_recursively(client=SnowflakeClient(), model=LLM_MODEL, max_tokens=LLM_MODEL_CONTEXT_WINDOWS, base_template_size=num_token_prompt_template, num_token_query=num_token_query, doc_entries=docs)
    
    for doc_batch in doc_entries_list:

        remaining_docs = doc_batch[:]  # docs still needing scores
        collected_scores = {}           # doc_id -> relevance_score
        attempt = 0

        while remaining_docs and attempt < MAX_RETRIES_PER_BATCH:
            attempt += 1

            prompt = build_prompt(prompt_template, query, remaining_docs)

            try:
                llm_response = get_llm_response(
                    client=SnowflakeClient(),
                    model=LLM_MODEL,
                    prompt=prompt,
                    response_format=json_schema,
                    temperature=LLM_TEMPERATURE,
                    max_tokens=LLM_MAX_TOKENS,
                )

                json_output = json.loads(llm_response)

                returned_docs = {
                    d["doc_id"]: d["relevance_score"]
                    for d in json_output["relevance_judgments"]
                }

                # Merge newly returned scores
                collected_scores.update(returned_docs)

                # Detect still-missing docs
                returned_ids = set(returned_docs.keys())
                remaining_docs = [
                    doc for doc in remaining_docs if doc["ID"] not in returned_ids
                ]

                if remaining_docs:
                    print(
                        f"Retry {attempt}/{MAX_RETRIES_PER_BATCH} | "
                        f"Missing {len(remaining_docs)} docs for query='{query}'"
                    )

            except Exception as e:
                print(
                    f"❌ LLM error on retry {attempt} "
                    f"(batch size={len(remaining_docs)})"
                )
                print(e)
                break

        # 🧩 Finalize batch (guaranteed coverage)
        for doc in doc_batch:
            doc_id = doc["ID"]
            query_doc_relevance_dict["relevance_judgments"].append({
                "doc_id": doc_id,
                "relevance_score": collected_scores.get(doc_id, 0.0)
            })

        # Log hard failures (after retries)
        if remaining_docs:
            print(
                f"FINAL MISSING after {MAX_RETRIES_PER_BATCH} retries "
                f"for query='{query}': {[d['ID'] for d in remaining_docs]}"
            )
  
    llm_results.append(query_doc_relevance_dict)    
    break  

    print(f"✓ Parsed JSON for query: {query}")

# #Save to file for future runs
# with open(OUTPUT_RETRIEVED_DOCS_RELEVENCE_FILE, "w", encoding="utf-8") as f:
#     json.dump(llm_results, f, indent=4)

In [ ]:
llm_results[0]

In [ ]:
llm_results.append(query_doc_relevance_dict)

In [ ]:
with open(OUTPUT_RETRIEVED_DOCS_RELEVENCE_FILE, "r", encoding="utf-8") as f:
        llm_results = json.load(f)

In [ ]:
from collections import defaultdict

merged_llm_results = {}

for item in llm_results:
    query = item["query_text"]
    judgments = item.get("relevance_judgments", [])

    if query not in merged_llm_results:
        merged_llm_results[query] = {
            "query_text": query,
            "relevance_judgments": []
        }

    merged_llm_results[query]["relevance_judgments"].extend(judgments)

# Convert back to list (same format as before)
llm_results_merged = list(merged_llm_results.values())


In [ ]:
query_retrived_docs_dict

In [ ]:
#find the doc with id 184442 and query a vegan dessert that require minimum time to prepare
[doc for doc in llm_results_merged[0]["relevance_judgments"]]


In [ ]:
query_retrived_docs_dict['a vegan dessert that require minimum time to prepare']

In [ ]:
[doc for doc in query_retrived_docs_dict['a vegan dessert that require minimum time to prepare'] if doc['ID'] == 184442]

In [ ]:

doc_id_initial = set([doc['ID'] for doc in query_retrived_docs_dict['a vegan dessert that require minimum time to prepare']])
doc_id_llm = set([doc['doc_id'] for doc in llm_results_merged[0]["relevance_judgments"]])

doc_id_initial - doc_id_llm

In [ ]:
first_batch = set([doc['ID'] for doc in doc_entries_list[0]])
second_batch = set([doc['ID'] for doc in doc_entries_list[1]])
entire_bath = first_batch.union(second_batch)
doc_id_initial - entire_bath 

In [ ]:
doc_entries_list[0]

In [ ]:
len(set([doc['ID'] for doc in query_retrived_docs_dict['easy baking']]))

In [ ]:
len(set([doc['doc_id'] for doc in llm_results_merged[34]["relevance_judgments"]]))

**Associate the relevancy score to each pair of (query, doc) for each top-k, model, config** 

In [ ]:
# Build lookup: relevance_map[query][doc_id] = relevance_score
relevance_map = {}

for item in llm_results_merged:
    query = item["query_text"]
    relevance_map[query] = {
        r["doc_id"]: r["relevance_score"]
        for r in item["relevance_judgments"]
    }

for k, model_dict in retrived_model_query_docs_dict.items():
    for model_name, config_dict in model_dict.items():
        for config_name, query_dict in config_dict.items():
            for query, docs in query_dict.items():

                # skip if query was not judged by LLM
                if query not in relevance_map:
                    continue

                for doc in docs:
                    doc_id = doc.get("ID")

                    # attach relevance score if available
                    doc["relevance_score"] = relevance_map[query].get(doc_id)

In [ ]:
retrived_model_query_docs_dict

In [ ]:
retrived_model_query_docs_dict[1]